# 第8章: ニューラルネット

第6章で取り組んだニュース記事のカテゴリ分類を題材として，ニューラルネットワークでカテゴリ分類モデルを実装する．なお，この章ではPyTorch, TensorFlow, Chainerなどの機械学習プラットフォームを活用せよ．

## 70. 単語ベクトルの和による特徴量

問題50で構築した学習データ，検証データ，評価データを行列・ベクトルに変換したい．例えば，学習データについて，すべての事例$x_i$の特徴ベクトル$\boldsymbol{x}_i$を並べた行列$X$と，正解ラベルを並べた行列（ベクトル）$Y$を作成したい．

$
X = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \dots \\ 
  \boldsymbol{x}_n \\ 
\end{pmatrix} \in \mathbb{R}^{n \times d},
Y = \begin{pmatrix} 
  y_1 \\ 
  y_2 \\ 
  \dots \\ 
  y_n \\ 
\end{pmatrix} \in \mathbb{N}^{n}
$

ここで，$n$は学習データの事例数であり，$\boldsymbol{x}_i \in \mathbb{R}^d$と$y_i \in \mathbb{N}$はそれぞれ，$i \in \{1, \dots, n\}$番目の事例の特徴量ベクトルと正解ラベルを表す．
なお，今回は「ビジネス」「科学技術」「エンターテイメント」「健康」の4カテゴリ分類である．$\mathbb{N}_{<4}$で$4$未満の自然数（$0$を含む）を表すことにすれば，任意の事例の正解ラベル$y_i$は$y_i \in \mathbb{N}_{<4}$で表現できる．
以降では，ラベルの種類数を$L$で表す（今回の分類タスクでは$L=4$である）．

$i$番目の事例の特徴ベクトル$\boldsymbol{x}_i$は，次式で求める．

$
\boldsymbol{x}_i = \frac{1}{T_i} \sum_{t=1}^{T_i} \mathrm{emb}(w_{i,t})
$

ここで，$i$番目の事例は$T_i$個の（記事見出しの）単語列$(w_{i,1}, w_{i,2}, \dots, w_{i,T_i})$から構成され，$\mathrm{emb}(w) \in \mathbb{R}^d$は単語$w$に対応する単語ベクトル（次元数は$d$）である．すなわち，$i$番目の事例の記事見出しを，その見出しに含まれる単語のベクトルの平均で表現したものが$\boldsymbol{x}_i$である．今回は単語ベクトルとして，問題60でダウンロードしたものを用いればよい．$300$次元の単語ベクトルを用いたので，$d=300$である．

$i$番目の事例のラベル$y_i$は，次のように定義する．

$
y_i = \begin{cases}
0 & (\mbox{記事}x_i\mbox{が「ビジネス」カテゴリの場合}) \\
1 & (\mbox{記事}x_i\mbox{が「科学技術」カテゴリの場合}) \\
2 & (\mbox{記事}x_i\mbox{が「エンターテイメント」カテゴリの場合}) \\
3 & (\mbox{記事}x_i\mbox{が「健康」カテゴリの場合}) \\
\end{cases}
$

なお，カテゴリ名とラベルの番号が一対一で対応付いていれば，上式の通りの対応付けでなくてもよい．

以上の仕様に基づき，以下の行列・ベクトルを作成し，ファイルに保存せよ．

+ 学習データの特徴量行列: $X_{\rm train} \in \mathbb{R}^{N_t \times d}$
+ 学習データのラベルベクトル: $Y_{\rm train} \in \mathbb{N}^{N_t}$
+ 検証データの特徴量行列: $X_{\rm valid} \in \mathbb{R}^{N_v \times d}$
+ 検証データのラベルベクトル: $Y_{\rm valid} \in \mathbb{N}^{N_v}$
+ 評価データの特徴量行列: $X_{\rm test} \in \mathbb{R}^{N_e \times d}$
+ 評価データのラベルベクトル: $Y_{\rm test} \in \mathbb{N}^{N_e}$

なお，$N_t, N_v, N_e$はそれぞれ，学習データの事例数，検証データの事例数，評価データの事例数である．


In [1]:
import gensim
import numpy as np

model = gensim.models.KeyedVectors.load_word2vec_format('GoogleNews-vectors-negative300.bin', binary=True)

datatype = ["train", "valid", "test"]
category = {"b":0, "t":1, "e":2, "m":3}

for i in range(3):
    x, y = [], []
    with open(f"NewsAggregatorDataset/{datatype[i]}.txt") as f:
        with open(f"NewsAggregatorDataset/x_{datatype[i]}.txt", "w") as fx:
            with open(f"NewsAggregatorDataset/y_{datatype[i]}.txt", "w") as fy:
                _ = f.readline() #1行目無視
                for line in f.read().splitlines():
                    tmp = line.split("\t") #tmp[0] :category, tmp[1:]:title
                    
                    x = (np.array([model[w] for w in tmp[1].split() if w in model]))
                    if np.sum(x) == 0:
                        x = np.zeros((1, 300))
                    fx.write(" ".join([str(i) for i in list(x.mean(axis=0))]) + "\n")
                    fy.write(str(category[tmp[0]]) + "\n")

## 71. 単層ニューラルネットワークによる予測

問題70で保存した行列を読み込み，学習データについて以下の計算を実行せよ．

$
\hat{\boldsymbol{y}}_1 = {\rm softmax}(\boldsymbol{x}_1 W), \\
\hat{Y} = {\rm softmax}(X_{[1:4]} W)
$

ただし，${\rm softmax}$はソフトマックス関数，$X_{[1:4]} \in \mathbb{R}^{4 \times d}$は特徴ベクトル$\boldsymbol{x}_1, \boldsymbol{x}_2, \boldsymbol{x}_3, \boldsymbol{x}_4$を縦に並べた行列である．

$
X_{[1:4]} = \begin{pmatrix} 
  \boldsymbol{x}_1 \\ 
  \boldsymbol{x}_2 \\ 
  \boldsymbol{x}_3 \\ 
  \boldsymbol{x}_4 \\ 
\end{pmatrix}
$

行列$W \in \mathbb{R}^{d \times L}$は単層ニューラルネットワークの重み行列で，ここではランダムな値で初期化すればよい（問題73以降で学習して求める）．なお，$\hat{\boldsymbol{y}}_1 \in \mathbb{R}^L$は未学習の行列$W$で事例$x_1$を分類したときに，各カテゴリに属する確率を表すベクトルである．
同様に，$\hat{Y} \in \mathbb{R}^{n \times L}$は，学習データの事例$x_1, x_2, x_3, x_4$について，各カテゴリに属する確率を行列として表現している．

In [2]:
import torch

class SLPNet(torch.nn.Module): #単層ニューラルネットワーククラス
    def __init__(self, input_size, output_size):
        super().__init__()
        self.fc = torch.nn.Linear(input_size, output_size, bias=False)
        torch.nn.init.normal_(self.fc.weight, 0.0, 1.0)  # 正規乱数で重みを初期化

    def forward(self, x):
        x = self.fc(x)
        return x

X_train = np.loadtxt("NewsAggregatorDataset/x_train.txt", delimiter=" ")
X_train = torch.tensor(X_train, dtype=torch.float32)

model = SLPNet(300, 4)

print(torch.softmax(model(X_train[:1]), dim=1))
print(torch.softmax(model.forward(X_train[:4]), dim=1))

tensor([[0.0825, 0.7444, 0.1701, 0.0029]], grad_fn=<SoftmaxBackward0>)
tensor([[0.0825, 0.7444, 0.1701, 0.0029],
        [0.1247, 0.0379, 0.8147, 0.0227],
        [0.5386, 0.0595, 0.2837, 0.1181],
        [0.1729, 0.1174, 0.4721, 0.2376]], grad_fn=<SoftmaxBackward0>)


## 72. 損失と勾配の計算

学習データの事例$x_1$と事例集合$x_1, x_2, x_3, x_4$に対して，クロスエントロピー損失と，行列$W$に対する勾配を計算せよ．なお，ある事例$x_i$に対して損失は次式で計算される．

$
l_i = - \log [\mbox{事例}x_i\mbox{が}y_i\mbox{に分類される確率}]
$

ただし，事例集合に対するクロスエントロピー損失は，その集合に含まれる各事例の損失の平均とする．


In [3]:
y_train = np.loadtxt("NewsAggregatorDataset/y_train.txt")
y_train = torch.tensor(y_train, dtype=torch.int64)
loss = torch.nn.CrossEntropyLoss()

l1 = loss(model(X_train[:1]), y_train[:1])
model.zero_grad()
l1.backward()
print(f"損失 : {l1:.4f}")
print (f"勾配 : {model.fc.weight.grad}")

print("\n")
l2 = loss(model(X_train[:4]), y_train[:4])
model.zero_grad()
l2.backward()
print(f"損失 : {l2:.4f}")
print (f"勾配 : {model.fc.weight.grad}")

損失 : 2.4945
勾配 : tensor([[ 1.6687e-02,  5.3151e-02, -2.7999e-03,  ...,  1.0584e-02,
         -7.2914e-02,  9.0903e-02],
        [-1.3540e-02, -4.3127e-02,  2.2718e-03,  ..., -8.5875e-03,
          5.9162e-02, -7.3759e-02],
        [-3.0944e-03, -9.8562e-03,  5.1920e-04,  ..., -1.9626e-03,
          1.3521e-02, -1.6857e-02],
        [-5.2791e-05, -1.6815e-04,  8.8576e-06,  ..., -3.3482e-05,
          2.3067e-04, -2.8758e-04]])


損失 : 1.4286
勾配 : tensor([[ 6.7104e-03,  1.6966e-02,  9.3049e-03,  ..., -1.7153e-02,
         -3.1547e-02,  1.9028e-02],
        [-2.9929e-03, -8.8860e-03, -1.7221e-03,  ..., -8.4937e-04,
          2.0667e-02, -1.9901e-02],
        [-3.8114e-03, -1.0906e-02, -5.2077e-03,  ...,  1.4672e-02,
          1.5162e-04,  3.4093e-03],
        [ 9.3887e-05,  2.8256e-03, -2.3752e-03,  ...,  3.3301e-03,
          1.0729e-02, -2.5366e-03]])


## 73. 確率的勾配降下法による学習

確率的勾配降下法（SGD: Stochastic Gradient Descent）を用いて，行列$W$を学習せよ．なお，学習は適当な基準で終了させればよい（例えば「100エポックで終了」など）．

In [4]:
from torch.utils.data import TensorDataset, DataLoader

dataset = TensorDataset(X_train, y_train)
loader = DataLoader(dataset, batch_size=1, shuffle=True)

model = SLPNet(300, 4)
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-1)

num_epochs = 10
print("epoch\tloss")
for epoch in range(num_epochs):
    sum_loss = 0
    for x, y in loader:
        y_pred = model(x)
        loss = loss_fn(y_pred, y)
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        sum_loss += loss
    print(f"{epoch+1}\t{sum_loss:.4f}")

epoch	loss
1	5741.3779
2	3817.6082
3	3500.2349
4	3338.0598
5	3228.5181
6	3162.4631
7	3122.0759
8	3079.0791
9	3038.4519
10	3020.4028


## 74. 正解率の計測

問題73で求めた行列を用いて学習データおよび評価データの事例を分類したとき，その正解率をそれぞれ求めよ．

In [5]:
def accuracy(pred, label):
    pred = np.argmax(pred.data.numpy(), axis=1)
    label = label.data.numpy()
    return (pred == label).mean()

X_valid = np.loadtxt("NewsAggregatorDataset/x_valid.txt", delimiter=" ")
X_valid = torch.tensor(X_valid, dtype=torch.float32)
y_valid = np.loadtxt("NewsAggregatorDataset/y_valid.txt")
y_valid = torch.tensor(y_valid, dtype=torch.int64)

pred = model(X_train)
print(f"正解率（学習データ）： {accuracy(pred, y_train)}")
pred = model(X_valid)
print(f"正解率（学習データ）： {accuracy(pred, y_valid)}")

正解率（学習データ）： 0.9060277049794084
正解率（学習データ）： 0.8854790419161677


## 75. 損失と正解率のプロット

問題73のコードを改変し，各エポックのパラメータ更新が完了するたびに，訓練データでの損失，正解率，検証データでの損失，正解率をグラフにプロットし，学習の進捗状況を確認できるようにせよ．

## 76. チェックポイント

問題75のコードを改変し，各エポックのパラメータ更新が完了するたびに，チェックポイント（学習途中のパラメータ（重み行列など）の値や最適化アルゴリズムの内部状態）をファイルに書き出せ．

## 77. ミニバッチ化

問題76のコードを改変し，$B$事例ごとに損失・勾配を計算し，行列$W$の値を更新せよ（ミニバッチ化）．$B$の値を$1, 2, 4, 8, \dots$と変化させながら，1エポックの学習に要する時間を比較せよ．

## 78. GPU上での学習

問題77のコードを改変し，GPU上で学習を実行せよ．

## 79. 多層ニューラルネットワーク

問題78のコードを改変し，バイアス項の導入や多層化など，ニューラルネットワークの形状を変更しながら，高性能なカテゴリ分類器を構築せよ．